# Práctica Calificada 02
## Predicción de tarifas de taxis

<span style="color:red">
Abrir el cuaderno en Google Colab y ejecutar las celdas nuevamente.
</span>.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
drive.mount('/gdrive')
%cd /gdrive/MyDrive/
#%ls

Mounted at /gdrive
/gdrive/MyDrive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

#### Importar data total

In [ ]:
dtypes = {
      'key': 'object',
      'fare_amount': 'float64',
      'pickup_datetime': 'object',
      'pickup_longitude': 'float64', 
      'pickup_latitude': 'float64',
      'dropoff_longitude': 'float64',
      'dropoff_latitude': 'float64',
      'passenger_count': 'int64',
      
    
}
df = pd.read_csv("train.csv", dtype=dtypes, nrows=1000000)
df = df[['key', 'pickup_datetime', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'passenger_count', 'fare_amount']]
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [ ]:
df.info()

### Primera revisión de la data completa

#### Valores duplicados

In [ ]:
%time
duplicateRowsDF = df.loc[df.duplicated(['key'], keep=False), :]
duplicateRowsDF

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count


In [ ]:
df['key'][55423855] =  '2014-12-12 11:33:00.00000016'

In [ ]:
duplicateRowsDF = df.loc[df.duplicated(['key'], keep=False), :]
duplicateRowsDF.shape[0]

#### Valores NaN

In [ ]:
valores_nan = df.isna().sum()
valores_nan

In [ ]:
df = df.dropna()

#### Coordenadas

Se analizará la data con el objetivo de eliminar las filas con coordenadas ubicadas en el océano.


In [ ]:
!pip install global-land-mask

     |████████████████████████████████| 1.8MB 5.1MB/s 


In [ ]:
from global_land_mask import globe as gl

In [ ]:
df.shape

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11084698 entries, 0 to 11084770
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 761.1+ MB


In [ ]:
df = df[df['pickup_latitude'].values > -90]
df = df[df['pickup_latitude'].values < 90]
df = df[df['dropoff_latitude'].values > -90]
df = df[df['dropoff_latitude'].values < 90]

df = df[df['pickup_longitude'].values > -180]
df = df[df['pickup_longitude'].values < 180]
df = df[df['dropoff_longitude'].values > -180]
df = df[df['dropoff_longitude'].values < 180]

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
3,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
4,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1
...,...,...,...,...,...,...,...,...
11084766,2009-08-27 18:39:00.00000067,11.3,2009-08-27 18:39:00 UTC,-73.980997,40.762858,-73.990503,40.736850,1
11084767,2015-05-17 00:27:40.0000005,5.0,2015-05-17 00:27:40 UTC,-73.974289,40.761883,-73.982964,40.756119,3
11084768,2009-02-03 00:28:08.0000001,10.2,2009-02-03 00:28:08 UTC,-73.999809,40.726766,-73.979344,40.749732,2
11084769,2014-09-22 09:00:00.00000019,16.5,2014-09-22 09:00:00 UTC,-73.977880,40.763550,-73.974020,40.743170,6


In [ ]:
pla = df[['pickup_latitude']].to_numpy()
plon = df[['pickup_longitude']].to_numpy()
dla = df[['dropoff_latitude']].to_numpy()
dlon = df[['dropoff_longitude']].to_numpy()

#gl.is_ocean(pla,plon)
#df.loc[gl.is_ocean(pla,plon), :]

df = df.drop(df.loc[gl.is_ocean(pla,plon)].index)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
3,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
4,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1
...,...,...,...,...,...,...,...,...
11084766,2009-08-27 18:39:00.00000067,11.3,2009-08-27 18:39:00 UTC,-73.980997,40.762858,-73.990503,40.736850,1
11084767,2015-05-17 00:27:40.0000005,5.0,2015-05-17 00:27:40 UTC,-73.974289,40.761883,-73.982964,40.756119,3
11084768,2009-02-03 00:28:08.0000001,10.2,2009-02-03 00:28:08 UTC,-73.999809,40.726766,-73.979344,40.749732,2
11084769,2014-09-22 09:00:00.00000019,16.5,2014-09-22 09:00:00 UTC,-73.977880,40.763550,-73.974020,40.743170,6


In [ ]:
# dfs[['pickup_latitude', 'pickup_longitude']] = dfs.apply(lambda x: ocean_values_to_nan(x['pickup_latitude'],x['pickup_longitude']), axis=1 )
# dfs[['dropoff_latitude', 'dropoff_longitude']] = dfs.apply(lambda x: ocean_values_to_nan(x['dropoff_latitude'],x['dropoff_longitude']) , axis=1)

In [ ]:
df.shape

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [ ]:
#df.to_csv('trainsampleprep.csv')

In [ ]:
df.head(-15)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
3,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00 UTC,-73.986430,40.760465,-73.988990,40.737075,1
4,2013-02-12 12:15:46.0000002,11.5,2013-02-12 12:15:46 UTC,-73.957954,40.779252,-73.961250,40.758787,1


#### Rango de valores

Ahora verificamos que los datos cumplan las restricciones de rango correspondientes.

In [ ]:
df.describe()

Observamos que el mínimo valor de la columna de tarifa (fare_amount) es -30. Necesitamos que los valores de la tarifa sean valores mayores o iguales a 0.

In [ ]:
df[df['fare_amount'] <= 0].count() /df.count()

In [ ]:
df = df[df['fare_amount'] > 0]

También notamos valores atípicos en la cantidad de pasajeros, por ejemplo un viaje con 208 pasajeros. De acuerdo a las leyes de tránsito de Nueva York como máximo pueden haber 6 personas. Por lo tanto, las filas con cantidades de pasajeros mayores a 6 serán eliminadas.

In [ ]:
( (df[df['passenger_count'] > 6].count() + df[df['passenger_count'] < 1.0].count()) / df.count() )* 100

In [ ]:
df = df[df['passenger_count'] <= 6]
df = df[df['passenger_count'] >= 1.0]

In [ ]:
df.describe()

Dividimos la columna 'pickup_datetime' en 2 nuevas columnas que representarán el día y el momento del día (mañana, tarde o noche) en que se produjo la parada del taxi

In [ ]:
df['day']=df['pickup_datetime'].apply(lambda x: pd.Timestamp(x).day_name())
df.head()

,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount,day
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,40.721319,-73.844311,40.712278,-73.841610,1,4.5,Monday
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,40.711303,-74.016048,40.782004,-73.979268,1,16.9,Tuesday
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,40.761270,-73.982738,40.750562,-73.991242,2,5.7,Thursday
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,40.733143,-73.987130,40.758092,-73.991567,1,7.7,Saturday
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,40.768008,-73.968095,40.783762,-73.956655,1,5.3,Tuesday


Convertimos la variable categórica 'dia' a una variable indicadora

In [ ]:
df['day'] = pd.factorize(df.day)[0]
#df = pd.get_dummies(df, columns=['day'])

In [ ]:
df.head()

,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount,day
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,40.721319,-73.844311,40.712278,-73.841610,1,4.5,0
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,40.711303,-74.016048,40.782004,-73.979268,1,16.9,1
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,40.761270,-73.982738,40.750562,-73.991242,2,5.7,2
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,40.733143,-73.987130,40.758092,-73.991567,1,7.7,3
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,40.768008,-73.968095,40.783762,-73.956655,1,5.3,1


Obtenemos la hora de la misma manera de 'pickup_datetime'

In [ ]:
df['hour']=df['pickup_datetime'].apply(lambda x: pd.Timestamp(x).hour)
df.head()

,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount,day,hour
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,40.721319,-73.844311,40.712278,-73.841610,1,4.5,0,17
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,40.711303,-74.016048,40.782004,-73.979268,1,16.9,1,16
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,40.761270,-73.982738,40.750562,-73.991242,2,5.7,2,0
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,40.733143,-73.987130,40.758092,-73.991567,1,7.7,3,4
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,40.768008,-73.968095,40.783762,-73.956655,1,5.3,1,7


Pasamos la columna hora (números) a categorías (mañana, tarde y noche)

In [ ]:
df['hour']=df['hour'].apply(lambda x: 'manana' if x > 5 and x < 12 else 'Tarde' if x >=12 and x < 18 else 'noche')
df.head()                          
    

,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount,day,hour
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,40.721319,-73.844311,40.712278,-73.841610,1,4.5,0,Tarde
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,40.711303,-74.016048,40.782004,-73.979268,1,16.9,1,Tarde
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,40.761270,-73.982738,40.750562,-73.991242,2,5.7,2,noche
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,40.733143,-73.987130,40.758092,-73.991567,1,7.7,3,noche
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,40.768008,-73.968095,40.783762,-73.956655,1,5.3,1,manana


Convertimos la columna hora de variables categóricas a indicadoras

In [ ]:
df['hour'] = pd.factorize(df.hour)[0]
df.head()  

,key,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,fare_amount,day,hour
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,40.721319,-73.844311,40.712278,-73.841610,1,4.5,0,0
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,40.711303,-74.016048,40.782004,-73.979268,1,16.9,1,0
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,40.761270,-73.982738,40.750562,-73.991242,2,5.7,2,1
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,40.733143,-73.987130,40.758092,-73.991567,1,7.7,3,1
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,40.768008,-73.968095,40.783762,-73.956655,1,5.3,1,2


#### Normalización

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfn = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
dfn.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0.096626,0.256750,0.094779,0.486964,0.509565,0.487987,0.520137,0.004808
1,0.095404,0.254083,0.093577,0.486967,0.509570,0.487992,0.520145,0.014423
2,0.193632,0.259417,0.189024,0.486962,0.509562,0.487989,0.520142,0.004808
3,0.841936,0.255000,0.810950,0.486965,0.509566,0.487990,0.520143,0.009615
4,0.384110,0.260083,0.370704,0.486959,0.509561,0.487990,0.520142,0.004808


In [ ]:
# dff = pd.DataFrame(af, columns = ['key','fare_amount','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count'])
# dff.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0.096626,0.256750,0.094779,0.486964,0.509565,0.487987,0.520137,0.004808
1,0.095404,0.254083,0.093577,0.486967,0.509570,0.487992,0.520145,0.014423
2,0.193632,0.259417,0.189024,0.486962,0.509562,0.487989,0.520142,0.004808
3,0.841936,0.255000,0.810950,0.486965,0.509566,0.487990,0.520143,0.009615
4,0.384110,0.260083,0.370704,0.486959,0.509561,0.487990,0.520142,0.004808
